In [2]:
!pip install --upgrade --quiet pinecone-client pinecone-text pinecone-notebooks

In [4]:
api_key="b896a336-07dd-4064-9a0c-f4fc8b092ff2"

In [3]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [5]:
import os
from pinecone import Pinecone,ServerlessSpec

In [6]:
index_name="hybrid-search-lagchain-pinecone"

In [9]:
pc=Pinecone(api_key=api_key)
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="dotproduct",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )

In [10]:
index=pc.Index(index_name)

In [11]:
## vector embedding 
## sparse matrix

In [13]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")

In [16]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [17]:
from pinecone_text.sparse import BM25Encoder

In [18]:
bm25_encoder=BM25Encoder().default()
bm25_encoder

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\AshishRawat\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [19]:
sentences=[
    "Hie , wlecome to the era of genai",
    "We are doing experimentation on Hybrid Search RAG with Pine cone",
    "The project look ammazing"
]

In [20]:
bm25_encoder.fit(sentences)

100%|██████████| 3/3 [00:00<00:00, 214.55it/s]


In [21]:
## store the values to josn
bm25_encoder.dump("bm25_encoder.json")

In [22]:
retriver=PineconeHybridSearchRetriever(embeddings=embeddings,
                              sparse_encoder=bm25_encoder,
                              index=index)

In [23]:
retriver

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x000002956E5C33A0>, index=<pinecone.data.index.Index object at 0x000002956C8FB580>)

In [24]:
retriver.add_texts(
    sentences
)

100%|██████████| 1/1 [00:04<00:00,  4.53s/it]


In [26]:
retriver.invoke("HOw is the project ?")

[Document(page_content='The project look ammazing'),
 Document(page_content='We are doing experimentation on Hybrid Search RAG with Pine cone'),
 Document(page_content='Hie , wlecome to the era of genai')]